# Pytorch Tutorial

This file is used as a reference to pytorch syntax.

Pytorch is a python framework for machine learning

- GPU-accelerated computations
- automatic differentiation
- modules for neural networks

This tutorial will teach you the fundamentals of operating on pytorch tensors and networks. You have already seen some things in recitation 0 which we will quickly review, but most of this tutorial is on mostly new or more advanced stuff.

For a worked example of how to build and train a pytorch network, see `pytorch-example.ipynb`.

For additional tutorials, see http://pytorch.org/tutorials/

In [1]:
import torch
import numpy as np
import torch.nn as nn

In [12]:
torch.__version__

'1.6.0+cu101'

## Tensors (review)

Tensors are the fundamental object for array data. The most common types you will use are `IntTensor` and `FloatTensor`.

In [13]:
# Create uninitialized tensor
x = torch.FloatTensor(2,3)
print(x)
# Initialize to zeros
x.zero_()
print(x)

tensor([[-8.6565e-06,  0.0000e+00,  3.7835e-44],
        [ 0.0000e+00,         nan,  2.0849e-01]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])


In [14]:
# Create from numpy array (seed for repeatability)
np.random.seed(123)
np_array = np.random.random((2,3))
print(torch.FloatTensor(np_array))
print(torch.from_numpy(np_array))

tensor([[0.6965, 0.2861, 0.2269],
        [0.5513, 0.7195, 0.4231]])
tensor([[0.6965, 0.2861, 0.2269],
        [0.5513, 0.7195, 0.4231]], dtype=torch.float64)


In [15]:
# Create random tensor (seed for repeatability)
torch.manual_seed(123)
x=torch.randn(2,3)
print(x)
# export to numpy array
x_np = x.numpy()
print(x_np)

tensor([[-0.1115,  0.1204, -0.3696],
        [-0.2404, -1.1969,  0.2093]])
[[-0.11146712  0.12036294 -0.3696345 ]
 [-0.24041797 -1.1969243   0.20926936]]


In [16]:
# special tensors (see documentation)
print(torch.eye(3))
print(torch.ones(2,3))
print(torch.zeros(2,3))
print(torch.arange(0,3))

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0, 1, 2])


All tensors have a `size` and `type`

In [17]:
x=torch.FloatTensor(3,4)
print(x.size())
print(x.type())

torch.Size([3, 4])
torch.FloatTensor


## Math, Linear Algebra, and Indexing (review)

Pytorch math and linear algebra is similar to numpy. Operators are overridden so you can use standard math operators (`+`,`-`, etc.) and expect a tensor as a result. See pytorch documentation for a complete list of available functions.

In [18]:
x = torch.arange(0.,5.)
print(torch.sum(x))
print(torch.sum(torch.exp(x)))
print(torch.mean(x))

tensor(10.)
tensor(85.7910)
tensor(2.)


Pytorch indexing is similar to numpy indexing. See pytorch documentation for details.

In [19]:
x = torch.rand(3,2)
print(x)
print(x[1,:])

tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])
tensor([0.3164, 0.4017])


## CPU and GPU

Tensors can be copied between CPU and GPU. It is important that everything involved in a calculation is on the same device. 

This portion of the tutorial may not work for you if you do not have a GPU available.

In [20]:
# create a tensor
x = torch.rand(3,2)
# copy to GPU
y = x.cuda()
# copy back to CPU
z = y.cpu()
# get CPU tensor as numpy array
# cannot get GPU tensor as numpy array directly
try:
    y.cpu().numpy()
except RuntimeError as e:
    print(e)

Operations between GPU and CPU tensors will fail. Operations require all arguments to be on the same device.

In [21]:
x = torch.rand(3,5)  # CPU tensor
y = torch.rand(5,4).cuda()  # GPU tensor
# try:
#     torch.mm(x,y)  # Operation between CPU and GPU fails. Okay, we're not actually going to run this because this casues weird side effects in Colab that break everything else : )
# except TypeError as e:
#     print(e)

Typical code should include `if` statements or utilize helper functions so it can operate with or without the GPU.

In [22]:
# Put tensor on CUDA if available
x = torch.rand(3,2)
if torch.cuda.is_available():
    x = x.to("cuda:0")
    print(x, x.dtype)
    
# Do some calculations
y = x ** 2 
print(y)

# Copy to CPU if on GPU
if y.is_cuda:
    y = y.cpu()
    print(y, y.dtype)

tensor([[0.0950, 0.5789],
        [0.9131, 0.0275],
        [0.1634, 0.3009]], device='cuda:0') torch.float32
tensor([[9.0180e-03, 3.3508e-01],
        [8.3382e-01, 7.5626e-04],
        [2.6693e-02, 9.0521e-02]], device='cuda:0')
tensor([[9.0180e-03, 3.3508e-01],
        [8.3382e-01, 7.5626e-04],
        [2.6693e-02, 9.0521e-02]]) torch.float32


A convenient method is `new`, which creates a new tensor on the same device as another tensor. It should be used for creating tensors whenever possible.

In [23]:
x1 = torch.rand(3,2)
x2 = x1.new(1,2)  # create cpu tensor
print(x2)
x1 = torch.rand(3,2).cuda()
x2 = x1.new(1,2)  # create cuda tensor
print(x2)

tensor([[-8.6559e-06,  0.0000e+00]])
tensor([[0.0090, 0.3351]], device='cuda:0')


Calculations executed on the GPU can be many times faster than numpy. However, numpy is still optimized for the CPU and many times faster than python `for` loops. Numpy calculations may be faster than GPU calculations for small arrays due to the cost of interfacing with the GPU.

In [24]:
from timeit import timeit
# Create random data
x = torch.rand(1000,64)
y = torch.rand(64,32)
number = 10000  # number of iterations

def square():
    z=torch.mm(x, y) # dot product (mm=matrix multiplication)

# Time CPU
print('CPU: {}ms'.format(timeit(square, number=number)*1000))
# Time GPU
x, y = x.cuda(), y.cuda()
print('GPU: {}ms'.format(timeit(square, number=number)*1000))

CPU: 670.0546240000449ms
GPU: 118.93357199994625ms


## Differentiation

Tensors provide automatic differentiation.

As you might know, previous versions of Pytorch used Variables, which were wrappers around tensors for differentiation. Starting with pytorch 0.4.0, this wrapping is done internally in the Tensor class and you can, and should, differentiate Tensors directly. However, it is possible that you walk on references to Variables, e.g. in your error messages.

What you need to remember :

- Tensors you are differentiating with respect to must have `requires_grad=True`
- Call `.backward()` on scalar variables you are differentiating
- To differentiate a vector, sum it first

In [25]:
# Create differentiable tensor
x = torch.tensor(torch.arange(0,4), requires_grad=False)
print(x.dtype)
# Calculate y=sum(x**2)
y = x**2
# Calculate gradient (dy/dx=2x)
y.sum().backward() # this breaks because requires_grad is FALSE for x, so subsequently for y as well. Try setting it to True
# Print values
print(x)
print(y)
print(x.grad)

torch.int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


RuntimeError: ignored

Differentiation accumulates gradients. This is sometimes what you want and sometimes not. **Make sure to zero gradients between batches if performing gradient descent or you will get strange results!**

In [26]:
# Create a variable
x=torch.tensor(torch.arange(0,4), requires_grad=True, dtype=torch.float)
# Differentiate
torch.sum(x**2).backward()
print(x.grad)
# Differentiate again (accumulates gradient)
torch.sum(x**2).backward()
print(x.grad)
# Zero gradient before differentiating
x.grad.data.zero_()
torch.sum(x**2).backward()
print(x.grad)

tensor([0., 2., 4., 6.])
tensor([ 0.,  4.,  8., 12.])
tensor([0., 2., 4., 6.])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Note that a Tensor with gradient cannot be exported to numpy directly :

In [27]:
x=torch.tensor(torch.arange(0,4), requires_grad=True, dtype=torch.float)
x.numpy() # raises an exception

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


RuntimeError: ignored

The reason is that pytorch remembers the graph of all computations to perform differenciation. To be integrated to this graph the raw data is wrapped internally to the Tensor class (like what was formerly a Variable). You can detach the tensor from the graph using the **.detach()** method, which returns a tensor with the same data but requires_grad set to False.

In [28]:
x=torch.tensor(torch.arange(0,4), requires_grad=True, dtype=torch.float)
y=x**2
z=y**2
z.detach().numpy()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


array([ 0.,  1., 16., 81.], dtype=float32)

In [29]:
# If you're tired to seeing the warnings above, another way to write it would be
x = torch.arange(0,4).float()
x.requires_grad = True
y = x**2
z = y**2
z.detach().numpy()

array([ 0.,  1., 16., 81.], dtype=float32)

Another reason to use this method is that updating the graph can use a lot of memory. If you are in a context where you have a differentiable tensor that you don't need to differentiate, think of detaching it from the graph.

## Neural Network Modules

Pytorch provides a framework for developing neural network modules. They take care of many things, the main one being wrapping and tracking a list of parameters for you.
You have several ways of building and using a network, offering different tradeoffs between freedom and simplicity.

torch.nn provides basic 1-layer nets, such as Linear (perceptron) and activation layers.

In [30]:
x = torch.arange(0,32).float() # note that we didn't need to set requires_grad=True on the input! 
net = torch.nn.Linear(32,10)
y = net(x)
print(y)

tensor([-11.2654,  -5.1798, -12.6108,  10.2352,  -0.8935, -16.3450,   0.8731,
          8.7567,  15.4684,   2.9802], grad_fn=<AddBackward0>)


All nn.Module objects are reusable as components of bigger networks ! That is how you build personnalized nets. The simplest way is to use the nn.Sequential class.

You can also create your own class that inherits n.Module. The forward method should precise what happens in the forward pass given an input. This enables you to precise behaviors more complicated than just applying layers one after another, if necessary.

In [31]:
# create a simple sequential network (`nn.Module` object) from layers (other `nn.Module` objects).
# Here a MLP with 2 layers and sigmoid activation.
net = torch.nn.Sequential(
    torch.nn.Linear(32,128),
    torch.nn.Sigmoid(),
    torch.nn.Linear(128,10))

In [32]:
# create a more customizable network module (equivalent here)
class MyNetwork(torch.nn.Module):
    # you can use the layer sizes as initialization arguments if you want to
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(input_size,hidden_size)
        self.layer2 = torch.nn.Sigmoid()
        self.layer3 = torch.nn.Linear(hidden_size,output_size)

    def forward(self, input_val):
        h = input_val
        h = self.layer1(h)
        h = self.layer2(h)
        h = self.layer3(h)
        return h

net = MyNetwork(32,128,10)

The network tracks parameters, and you can access them through the **parameters()** method, which returns a python generator.

In [33]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0080, -0.1092, -0.0599,  ..., -0.1456,  0.0865,  0.1176],
        [-0.1304,  0.0644, -0.1766,  ..., -0.0393, -0.0157, -0.1341],
        [ 0.0140, -0.1288, -0.1243,  ..., -0.1510,  0.0714, -0.0168],
        ...,
        [ 0.1721,  0.0633,  0.0242,  ...,  0.1268,  0.0177, -0.1755],
        [-0.1404, -0.0018, -0.1066,  ...,  0.0408, -0.0123, -0.1635],
        [ 0.0330,  0.1358, -0.1656,  ..., -0.0513, -0.0271, -0.0602]],
       requires_grad=True)
Parameter containing:
tensor([ 0.1677,  0.0115, -0.0710, -0.0480,  0.0504,  0.0364,  0.1576, -0.0507,
         0.1083, -0.1162, -0.0565, -0.0122,  0.0131,  0.0323, -0.1086,  0.1145,
         0.1153,  0.1578, -0.1741, -0.0879, -0.1259,  0.1038, -0.1328,  0.1236,
         0.0834,  0.1593,  0.1020, -0.1573, -0.0994, -0.0793,  0.0433, -0.0659,
        -0.0446, -0.1097, -0.0849, -0.1697, -0.1460,  0.1269,  0.0598,  0.0937,
         0.1518, -0.1610,  0.0869, -0.1608, -0.1033,  0.0641, -0.0692, -0.1414,
         0.1670

Parameters are of type Parameter, which is basically a wrapper for a tensor. How does pytorch retrieve your network's parameters ? They are simply all the attributes of type Parameter in your network. Moreover, if an attribute is of type nn.Module, its own parameters are added to your network's parameters ! This is why, when you define a network by adding up basic components such as nn.Linear, you should never have to explicitely define parameters.

However, if you are in a case where no pytorch default module does what you need, you can define parameters explicitely (this should be rare). For the record, let's build the previous MLP with personnalized parameters.

In [34]:
class MyNetworkWithParams(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super(MyNetworkWithParams,self).__init__()
        self.layer1_weights = nn.Parameter(torch.randn(input_size,hidden_size))
        self.layer1_bias = nn.Parameter(torch.randn(hidden_size))
        self.layer2_weights = nn.Parameter(torch.randn(hidden_size,output_size))
        self.layer2_bias = nn.Parameter(torch.randn(output_size))
        
    def forward(self,x):
        h1 = torch.matmul(x,self.layer1_weights) + self.layer1_bias
        h1_act = torch.max(h1, torch.zeros(h1.size())) # ReLU
        output = torch.matmul(h1_act,self.layer2_weights) + self.layer2_bias
        return output

net = MyNetworkWithParams(32,128,10)

Parameters are useful in that they are meant to be all the network's weights that will be optimized during training. If you were needing to use a tensor in your computational graph that you want to remain constant, just define it as a regular tensor.

## Training

In [35]:
net = MyNetwork(32,128,10)

The nn.Module also provides loss functions, such as cross-entropy.

In [36]:
x = torch.tensor([np.arange(32), np.zeros(32),np.ones(32)]).float()
y = torch.tensor([0,3,9])
criterion = nn.CrossEntropyLoss()

output = net(x)
loss = criterion(output,y)
print(loss)

tensor(2.4213, grad_fn=<NllLossBackward>)


nn.CrossEntropyLoss does both the softmax and the actual cross-entropy : given $output$ of size $(n,d)$ and $y$ of size $n$ and values in $0,1,...,d-1$, it computes $\sum_{i=0}^{n-1}log(s[i,y[i]])$ where $s[i,j] = \frac{e^{output[i,j]}}{\sum_{j'=0}^{d-1}e^{output[i,j']}}$

You can also compose nn.LogSoftmax and nn.NLLLoss to get the same result. Note that all these use the log-softmax rather than the softmax, for stability in the computations.

In [37]:
# equivalent
criterion2 = nn.NLLLoss()
sf = nn.LogSoftmax()
output = net(x)
loss = criterion(sf(output),y)
loss

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  """


tensor(2.4213, grad_fn=<NllLossBackward>)

Now, to perform the backward pass, just execute **loss.backward()** ! It will update gradients in all differentiable tensors in the graph, which in particular includes all the network parameters.

In [38]:
loss.backward()

# Check that the parameters now have gradients
for param in net.parameters():
    print(param.grad)

tensor([[ 0.0003,  0.0003,  0.0003,  ...,  0.0003,  0.0003,  0.0003],
        [-0.0037, -0.0037, -0.0037,  ..., -0.0037, -0.0037, -0.0037],
        [-0.0078, -0.0078, -0.0078,  ..., -0.0074, -0.0074, -0.0074],
        ...,
        [-0.0040, -0.0040, -0.0040,  ..., -0.0045, -0.0046, -0.0046],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038],
        [-0.0029, -0.0047, -0.0066,  ..., -0.0569, -0.0587, -0.0606]])
tensor([-9.4054e-04, -6.1805e-03, -1.2012e-02,  9.7801e-03,  1.1470e-03,
         6.0123e-03,  1.8151e-03,  3.6971e-03, -8.1812e-03, -3.9692e-03,
         6.5300e-04, -9.2533e-03, -1.8172e-03,  2.0503e-03,  4.2377e-04,
         4.4279e-03, -4.0830e-03, -1.2903e-02, -7.1773e-03,  9.7517e-04,
         5.7179e-03,  8.3892e-03,  7.8087e-04, -1.7488e-03,  2.3776e-03,
        -4.1248e-03, -2.3352e-03, -6.4370e-04,  1.1360e-03, -4.4238e-03,
        -2.9335e-03, -5.9201e-03,  1.1329e-04, -4.7018e-03,  1.9633e-03,
         4.3207e-04,  3.1228e-03,  4.5239e-03,  2.5811e

In [39]:
# if I forward prop and backward prop again, gradients accumulate :
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

# you can remove this behavior by reinitializing the gradients in your network parameters :
net.zero_grad()
output = net(x)
loss = criterion(output,y)
loss.backward()
for param in net.parameters():
    print(param.grad)

tensor([[ 0.0007,  0.0007,  0.0007,  ...,  0.0007,  0.0007,  0.0007],
        [-0.0073, -0.0073, -0.0073,  ..., -0.0073, -0.0073, -0.0073],
        [-0.0156, -0.0156, -0.0155,  ..., -0.0148, -0.0148, -0.0148],
        ...,
        [-0.0079, -0.0080, -0.0080,  ..., -0.0091, -0.0091, -0.0092],
        [ 0.0075,  0.0075,  0.0075,  ...,  0.0075,  0.0075,  0.0075],
        [-0.0058, -0.0095, -0.0132,  ..., -0.1137, -0.1174, -0.1212]])
tensor([-1.8811e-03, -1.2361e-02, -2.4024e-02,  1.9560e-02,  2.2940e-03,
         1.2025e-02,  3.6301e-03,  7.3943e-03, -1.6362e-02, -7.9383e-03,
         1.3060e-03, -1.8507e-02, -3.6345e-03,  4.1006e-03,  8.4755e-04,
         8.8559e-03, -8.1659e-03, -2.5805e-02, -1.4355e-02,  1.9503e-03,
         1.1436e-02,  1.6778e-02,  1.5617e-03, -3.4975e-03,  4.7552e-03,
        -8.2496e-03, -4.6704e-03, -1.2874e-03,  2.2720e-03, -8.8477e-03,
        -5.8670e-03, -1.1840e-02,  2.2658e-04, -9.4036e-03,  3.9265e-03,
         8.6415e-04,  6.2455e-03,  9.0478e-03,  5.1621e

We did backpropagation, but still didn't perform gradient descent. Let's define an optimizer on the network parameters.

In [40]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

print("Parameters before gradient descent :")
for param in net.parameters():
    print(param)

optimizer.step()

print("Parameters after gradient descent :")
for param in net.parameters():
    print(param)

Parameters before gradient descent :
Parameter containing:
tensor([[-0.1304,  0.0003, -0.1563,  ..., -0.1631, -0.0063, -0.1529],
        [-0.0755,  0.0227, -0.1221,  ...,  0.0014, -0.0819,  0.0647],
        [-0.0004,  0.1718,  0.0688,  ..., -0.0230, -0.0616,  0.1032],
        ...,
        [-0.0620,  0.1712, -0.0744,  ..., -0.1489,  0.0763, -0.0951],
        [-0.1070,  0.0719,  0.1619,  ...,  0.1180,  0.0759,  0.0698],
        [-0.1238,  0.1715, -0.1681,  ...,  0.0952,  0.0341, -0.0457]],
       requires_grad=True)
Parameter containing:
tensor([-0.0309,  0.0196,  0.0274, -0.1702, -0.0458,  0.0352, -0.1366,  0.1607,
        -0.1137, -0.0695,  0.0403,  0.0866, -0.0381, -0.0676, -0.1215,  0.1454,
         0.1548,  0.0473,  0.0093, -0.0777,  0.1307,  0.0738,  0.0428, -0.0367,
         0.1464,  0.1611,  0.0209,  0.0431, -0.1234, -0.0167,  0.1560, -0.0413,
        -0.1622,  0.1103, -0.0161, -0.0406, -0.0504,  0.0779,  0.0155,  0.0946,
         0.0614, -0.1548,  0.0500, -0.1348,  0.0805, -0.05

In [41]:
# In a training loop, we should perform many GD iterations.
n_iter = 1000
for i in range(n_iter):
    optimizer.zero_grad() # equivalent to net.zero_grad()
    output = net(x)
    loss = criterion(output,y)
    loss.backward()
    optimizer.step()
    print(loss)

tensor(2.2979, grad_fn=<NllLossBackward>)
tensor(2.1852, grad_fn=<NllLossBackward>)
tensor(2.0829, grad_fn=<NllLossBackward>)
tensor(1.9909, grad_fn=<NllLossBackward>)
tensor(1.9084, grad_fn=<NllLossBackward>)
tensor(1.8343, grad_fn=<NllLossBackward>)
tensor(1.7671, grad_fn=<NllLossBackward>)
tensor(1.7059, grad_fn=<NllLossBackward>)
tensor(1.6498, grad_fn=<NllLossBackward>)
tensor(1.5982, grad_fn=<NllLossBackward>)
tensor(1.5504, grad_fn=<NllLossBackward>)
tensor(1.5062, grad_fn=<NllLossBackward>)
tensor(1.4650, grad_fn=<NllLossBackward>)
tensor(1.4266, grad_fn=<NllLossBackward>)
tensor(1.3907, grad_fn=<NllLossBackward>)
tensor(1.3570, grad_fn=<NllLossBackward>)
tensor(1.3254, grad_fn=<NllLossBackward>)
tensor(1.2957, grad_fn=<NllLossBackward>)
tensor(1.2677, grad_fn=<NllLossBackward>)
tensor(1.2412, grad_fn=<NllLossBackward>)
tensor(1.2162, grad_fn=<NllLossBackward>)
tensor(1.1924, grad_fn=<NllLossBackward>)
tensor(1.1700, grad_fn=<NllLossBackward>)
tensor(1.1486, grad_fn=<NllLossBac

In [42]:
output = net(x)
print(output)
print(y)

tensor([[ 7.7690, -1.7035, -1.9214,  0.7850, -2.3263, -2.1545, -2.1495, -2.4258,
         -2.2692,  3.8586],
        [ 0.3468, -1.5723, -1.6643,  5.7087, -1.5293, -1.7218, -1.5148, -1.6034,
         -1.5335,  3.1441],
        [ 1.7902, -1.7117, -1.7323,  2.9577, -1.6445, -1.7842, -1.7376, -1.8327,
         -1.7686,  5.6042]], grad_fn=<AddmmBackward>)
tensor([0, 3, 9])


Now you know how to train a network ! For a complete training check the pytorch_example notebook.

## Saving and Loading

In [43]:
# get dictionary of keys to weights using `state_dict`
net = torch.nn.Sequential(
    torch.nn.Linear(28*28,256),
    torch.nn.Sigmoid(),
    torch.nn.Linear(256,10))
print(net.state_dict().keys())

odict_keys(['0.weight', '0.bias', '2.weight', '2.bias'])


In [44]:
# save a dictionary
torch.save(net.state_dict(),'test.t7')
# load a dictionary
net.load_state_dict(torch.load('test.t7'))

<All keys matched successfully>

## Common issues to look out for

### Type mismatch

In [47]:
net = nn.Linear(4,2)
x = torch.tensor([1,2,3,4])
y = net(x)
print(y)

RuntimeError: ignored

In [48]:
x = x.float()
x = torch.tensor([1.,2.,3.,4.])

In [49]:
x = 2* torch.ones(2,2)
y = 3* torch.ones(2,2)
print(x * y)
print(x.matmul(y))

tensor([[6., 6.],
        [6., 6.]])
tensor([[12., 12.],
        [12., 12.]])


In [50]:
x = torch.ones(4,5).long()
y = torch.arange(5)
print(x+y)
y = torch.arange(4).view(-1,1)
print(x+y)
y = torch.arange(4)
print(x+y) # exception

tensor([[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]])
tensor([[1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4]])


RuntimeError: ignored

In [51]:
# A note here: You will encounter the following exception a number of times:
torch.ones(1, 1) + torch.arange(1) # RuntimeError: expected device cpu and dtype Float but got device cpu and dtype Long
#This is because you can't do operations b/n long and float. to fix, either convert both to long or both to float.
torch.ones(1, 1) + torch.arange(1).float() # Works

tensor([[1.]])

In [52]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x)
print(x.t())
print(x.view(3,2))

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([[1, 4],
        [2, 5],
        [3, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [3]:
net = nn.Sequential(nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,2048),nn.ReLU(),
                   nn.Linear(2048,120))
x = torch.ones(256,2048)
y = torch.zeros(256).long()
net.cuda()
x = x.cuda()
y = y.cuda()
crit=nn.CrossEntropyLoss()
out = net(x)
loss = crit(out,y)
loss.backward()

In [4]:
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))
    
            
    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out

In [5]:
class MyNet(nn.Module):
    def __init__(self,n_hidden_layers):
        super(MyNet,self).__init__()
        self.n_hidden_layers=n_hidden_layers
        self.final_layer = nn.Linear(128,10)
        self.act = nn.ReLU()
        self.hidden = []
        for i in range(n_hidden_layers):
            self.hidden.append(nn.Linear(128,128))
        self.hidden = nn.ModuleList(self.hidden)
            
    def forward(self,x):
        h = x
        for i in range(self.n_hidden_layers):
            h = self.hidden[i](h)
            h = self.act(h)
        out = self.final_layer(h)
        return out